In [17]:
import pygame
import numpy as np
import random

# Constants
WIDTH = 800
HEIGHT = 600
CELL_SIZE = 10
NUM_PLATES = 8
PLATE_SPEED = 1
PLATE_FORCE = 0.05
MANTLE_FORCE = 0.01
CRUST_THICKNESS = 5
CRUST_TYPES = ['oceanic', 'continental']
CRUST_COLORS = {'oceanic': (0, 0, 255), 'continental': (128, 128, 128)}
CRUST_THICKNESS_RATIOS = {'oceanic': 0.8, 'continental': 1.0}

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Continent Shift Simulation')

class Plate:
    def __init__(self, position, velocity, orientation, crust_type):
        self.position = position
        self.velocity = velocity
        self.orientation = orientation
        self.crust_type = crust_type

    def update_position(self, grid_shape):
        self.position += self.velocity
        self.position %= grid_shape

    def update_orientation(self):
        self.orientation += np.random.uniform(-0.1, 0.1)

def create_grid(height, width):
    grid = np.zeros((height // CELL_SIZE, width // CELL_SIZE), dtype=[('type', 'U11'), ('height', 'f4')])
    for y in range(grid.shape[0]):
        for x in range(grid.shape[1]):
            grid[y, x]['type'] = random.choice(['continental', 'oceanic'])
            grid[y, x]['height'] = np.random.uniform(-1, 1)
    return grid

def create_plates(num_plates, grid_shape):
    plates = []
    grid_width = grid_shape[1] * CELL_SIZE
    grid_height = grid_shape[0] * CELL_SIZE
    rows = int(np.sqrt(num_plates))
    cols = num_plates // rows

    dx = grid_width // cols
    dy = grid_height // rows

    k = 0
    for k in range(num_plates):
        if k >= num_plates:
            break

        plate = Plate(
            position=np.array([np.random.randint(0, WIDTH )* CELL_SIZE,
                               np.random.randint(0, HEIGHT)* CELL_SIZE], dtype=float),
            velocity=np.array([np.random.uniform(-1, 1), np.random.uniform(-1, 1)]),
            orientation=np.random.uniform(0, 2 * np.pi),
            crust_type=CRUST_TYPES[k % len(CRUST_TYPES)]
        )
        plates.append(plate)
        k += 1
    return plates



def compute_forces(plates):
    forces = []
    for plate in plates:
        boundary_forces = np.zeros(2)
        for neighbor in plates:
            if not np.all(neighbor.position == plate.position):
                distance = np.linalg.norm(neighbor.position - plate.position)
                direction = (neighbor.position - plate.position) / distance
                if distance < CELL_SIZE * 2:
                    if plate.crust_type == neighbor.crust_type:
                        boundary_forces -= PLATE_FORCE * direction
                    else:
                        boundary_forces += PLATE_FORCE * direction

        mantle_forces = np.zeros(2)
        for other in plates:
            if not np.all(other.position == plate.position):
                distance = np.linalg.norm(other.position - plate.position)
                direction = (other.position - plate.position) / distance
                mantle_forces += MANTLE_FORCE * direction

        total_forces = boundary_forces + mantle_forces
        forces.append(total_forces)
    return forces

def update_plates(plates, forces, grid):
    for plate, force in zip(plates, forces):
        plate.velocity += force
        plate.velocity *= PLATE_SPEED / np.linalg.norm(plate.velocity)
        plate.update_position(grid.shape)

        plate.update_orientation()

        update_grid(grid, plate)

def update_grid(grid, plate):
    x, y = (plate.position / CELL_SIZE).astype(int)
    r = CRUST_THICKNESS_RATIOS[plate.crust_type]

    for i in range(-CRUST_THICKNESS, CRUST_THICKNESS + 1):
        for j in range(-CRUST_THICKNESS, CRUST_THICKNESS + 1):
            if i**2 + j**2 < CRUST_THICKNESS**2:
                nx, ny = x + i, y + j
                if 0 <= nx < grid.shape[1] and 0 <= ny < grid.shape[0]:
                    height = grid[ny, nx]['height']
                    if grid[ny, nx]['type'] == 'oceanic' and plate.crust_type == 'continental':
                        grid[ny, nx]['type'] = 'continental'
                        grid[ny, nx]['height'] = height * r
                    elif grid[ny, nx]['type'] == 'continental' and plate.crust_type == 'oceanic':
                        grid[ny, nx]['type'] = 'oceanic'
                        grid[ny, nx]['height'] = height / r

                        
def draw_grid(screen, grid):
    for y in range(grid.shape[0]):
        for x in range(grid.shape[1]):
            color = CRUST_COLORS[grid[y, x]['type']]
            height = int((grid[y, x]['height'] + 1) * 127.5)
            height = np.clip(height, 0, 255)  # Ensure the height value is within a valid range
            pygame.draw.rect(screen, (height, height, height), (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)

            
PLATE_RADIUS = 50
def draw_plates(screen, plates):
    for plate in plates:
        x, y = plate.position
        pygame.draw.circle(screen, (100, 0, 0), (int(x), int(y)), PLATE_RADIUS)
        dx, dy = PLATE_RADIUS * np.array([np.cos(plate.orientation), np.sin(plate.orientation)])
        pygame.draw.line(screen, (0, 0, 0), (int(x), int(y)), (int(x + dx), int(y + dy)), 2)

def main():
    grid = create_grid(HEIGHT, WIDTH)
    plates = create_plates(NUM_PLATES, grid.shape)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        screen.fill((255, 255, 255))

        forces = compute_forces(plates)
        update_plates(plates, forces, grid)

        draw_grid(screen, grid)
        draw_plates(screen, plates)

        pygame.display.update()
#         pygame.time.delay(20)  # Add a small delay to control the simulation speed

    pygame.quit()

if __name__ == "__main__":
    main()


OverflowError: cannot convert float infinity to integer

In [15]:
grid = create_grid(HEIGHT, WIDTH)
grid.shape

(120, 160)

In [3]:
create_grid(HEIGHT, WIDTH)

array([[('oceanic',  0.543415  ), ('oceanic', -0.40720978),
        ('oceanic',  0.5589549 ), ..., ('oceanic',  0.86144197),
        ('oceanic',  0.3101514 ), ('oceanic',  0.11521863)],
       [('oceanic',  0.11623217), ('oceanic',  0.46968526),
        ('oceanic',  0.86358285), ..., ('oceanic', -0.761267  ),
        ('oceanic', -0.75305825), ('oceanic',  0.70118004)],
       [('oceanic',  0.03524515), ('oceanic',  0.41268682),
        ('oceanic',  0.5606515 ), ..., ('oceanic', -0.2806636 ),
        ('oceanic',  0.16773456), ('oceanic', -0.14990485)],
       ...,
       [('continental', -0.34859112), ('continental',  0.78958964),
        ('continental',  0.01597658), ..., ('continental',  0.44047922),
        ('continental',  0.24423753), ('continental', -0.2575825 )],
       [('continental', -0.7914123 ), ('continental',  0.6467954 ),
        ('continental', -0.40807518), ..., ('continental',  0.63186026),
        ('continental', -0.66900307), ('continental', -0.71210957)],
       [('